# Quote Generation/Retrieval

Initial quotes were scraped from one webpage. But that would lead to quite a lot of repitition and kindof a boring bot. So...

### 2 Issues with Index retrieved/Generated Quotes:
**Goal:** To have ~100 quotes to have tweeted on a rolling basis since this is a quarter of a year and will not likely to be easily repeated.

### Probability of Repeating a Quote Exactly 2 Weeks Later

\begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{N - 1}{N} \right)^{\frac{14}{m}}
\end{equation*}

 Substituting in the values:

 \begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{99}{100} \right)^{14}
\end{equation*}

 Calculating this gives:

 \begin{equation*}
P(\text{repeated in 2 weeks}) \approx 0.135
\end{equation*}

Personally, I'd like it to be under 10% for a longer period of time (increases probability)

Tweaking the numbers a bit 
N = 200
days = 21

\begin{equation*}
P(\text{repeated in 3 weeks}) \approx  0.0980
\end{equation*}

The following should be conditional 1) That the quote generated has confidence that the quote was written by Ovid then 2) That the quote is not a variation of a quote already in DB.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import openai
import pandas as pd

# import utils
import configparser
from pprint import pprint
from llama_index import SimpleDirectoryReader


import os
import openai


import jupyter_black

jupyter_black.load()
# Access values from the sections

import sys

sys.path.append("../app")
import conn_utils

OPENAI_API_KEY = conn_utils.get_open_ai_key("../config.ini")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
directory = "index_store"

In [2]:
!ls ./../data

Amours.txt                 MetamorphosesI_VII.txt
Fasti.txt                  MetamorphosesVIII_XV.txt
Heroides.txt               MetamorphosesofPublius.txt
LastPoems.txt              RemediaAmoris.txt
LoversAssistant.txt


In [3]:
df = pd.read_json("./../ovid_quotes.json")

DATA_DIR = "./../data"
files = [
    "RemediaAmoris.txt",
    "Heroides.txt",
    "Amours.txt",
    "Fasti.txt",
    "MetamorphosesI_VII.txt",
    "MetamorphosesVIII_XV.txt",
    "MetamorphosesofPublius.txt",
    "LoversAssistant.txt",
    "LastPoems.txt",
]
docs = [f"{DATA_DIR}/{file}" for file in files]
documents = SimpleDirectoryReader(input_files=docs).load_data()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.create_documents(docs)
directory = "index_store"
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local("index_store", OpenAIEmbeddings())
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [4]:
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

response = qa_interface(
    f"""
I am a big fan of ovid. 
Please recommend 10 memorable quotes to me along with the source document they were taken from.

Do NOT include quotes I already have:
{df["Quote"].tolist()}
"""
)

print(response["result"])

Here are 10 memorable quotes from Ovid's works:

1. "Love yields to business. If you seek a way out of love, be busy; you'll be safe then." - Metamorphoses of Publius
2. "The gods behold all righteous actions." - Metamorphoses of Publius
3. "There is a god within us." - Metamorphoses of Publius
4. "The mind, conscious of rectitude, laughed to scorn the falsehood of report." - Metamorphoses of Publius
5. "Every lover is a soldier." - Metamorphoses of Publius
6. "Resist beginnings; the remedy comes too late when the disease has gained strength by long delays." - Remedia Amoris
7. "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind." - Metamorphoses of Publius
8. "We take no pleasure in permitted joys. But what's forbidden is more keenly sought." - Metamorphoses of Publius
9. "Who is allowed to sin, sins less." - Metamorphoses of Publius
10. "Time flies, death urges, heaven's revolving wheel turns swift, and nothing suffers long." - Metamorphos

#### Authorship authentication

**[Who Wrote it and Why?
Prompting Large-Language Models for Authorship Verification](https://arxiv.org/pdf/2310.08123.pdf)**

In [5]:
# LLM Authorship Attribution
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


template = """Task: On a scale of 0 to 1, with 0 indicating low confidence
and 1 indicating high confidence, please provide a general
assessment of the likelihood that given text 
written by the same author as the provided reference. Your answer should reflect a
moderate level of strictness in scoring. Here are some
relevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma,
parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.
Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms
such as OMG, Abbreviations without punctuation marks
such as Mr Rochester vs. Mr. Rochester,Unusual
abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevant
variables and devise a plan to solve the problem. Then,
carry out the plan and solve the problem step by step.
9. One (or both) of the texts is written by the famous Latin author "Ovid"
Finally, show the confidence score.

The following are all quotes by Ovid for reference:
'Love is a thing full of anxious fears.',
 'Now are fields of corn where Troy once stood.',
 "We're slow to believe what wounds us.",
 "The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”",
 "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'The gods behold all righteous actions.',
 'There is a god within us.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'Every lover is a soldier.',
 'Let the man who does not wish to be idle fall in love!',
 'Far away be that fate!',
 'They bear punishment with equanimity who have earned it.',
 "We take no pleasure in permitted joys. But what's forbidden is more keenly sought.",
 'Who is allowed to sin, sins less.' """
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
# chain.invoke({"text": ""})

In [6]:
# Extract Quotes from LLM Authorshipp Attribution
import pandas as pd
import spacy
import re

nlp = spacy.load("en_core_web_sm")
input_string = response["result"]


def get_quotes(input_string) -> List[str]:
    """Extract quotes from a string that has been created by an LLM string prompt."""
    # Extract the lines that start with a number
    lines_with_stripped_numbers = [
        re.sub(r"^\d+\.\s*", "", line.strip())
        for line in input_string.splitlines()
        if re.match(r"^\d+\.", line)
    ]

    # Print the extracted lines
    quotes = []
    pattern2 = r'"([^"]+)"\s*-\s*(.+)'

    for line in lines_with_stripped_numbers:
        match = re.search(pattern2, line)
        q = match.group(1)
        w = match.group(2)
        quotes.append((q, w))
    return quotes


quotes = get_quotes(response["result"])
quotes

[("Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
  'Metamorphoses of Publius'),
 ('The gods behold all righteous actions.', 'Metamorphoses of Publius'),
 ('There is a god within us.', 'Metamorphoses of Publius'),
 ('The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
  'Metamorphoses of Publius'),
 ('Every lover is a soldier.', 'Metamorphoses of Publius'),
 ('Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
  'Remedia Amoris'),
 ("Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
  'Metamorphoses of Publius'),
 ("We take no pleasure in permitted joys. But what's forbidden is more keenly sought.",
  'Metamorphoses of Publius'),
 ('Who is allowed to sin, sins less.', 'Metamorphoses of Publius'),
 ("Time flies, death urges, heaven's revolving wheel turns swift, and nothing suffers long.",
  'Metamorphoses of Publius')]

### Score New Quotes

In [7]:
def score_quotes(quotes=None):
    """Score a list of quotes that have been generated by an LLM string prompt.

    quotes example:
    [('Dripping water hollows out stone, not through force but through persistence.',
    'Metamorphoses'),
    """
    if quotes is None:
        quotes = []

    # Define the regex pattern
    pattern = r"(\d+\.\d+)"

    # Get the quote string (not the work)
    quotes_text = [q[0] for q in quotes]
    quotes_work = [q[1] for q in quotes]

    # Extract the scores
    scores = []
    score_reasons = []
    for q in quotes_text:
        r = chain.invoke({"text": q})
        r = " ".join(r)
        score_reasons.append(r)
        match = re.search(pattern, r)
        score = match[0] if match else -1
        scores.append(score)
    return list(zip(*(scores, score_reasons, quotes_text, quotes_work)))


# Example usage
scored_quotes = score_quotes(quotes)
scored_quotes_df = pd.DataFrame(
    scored_quotes, columns=["score", "reason", "quote", "work"]
)
scored_quotes_df  # ["score"]

,score,reason,quote,work
0,0.9,Based on the provided quote the likelihood tha...,Love yields to business. If you seek a way out...,Metamorphoses of Publius
1,0.9,Based on the provided quote I would assess the...,The gods behold all righteous actions.,Metamorphoses of Publius
2,0.8,Confidence Score: 0.8\n\nExplanation: The writ...,There is a god within us.,Metamorphoses of Publius
3,0.9,Based on the provided quote by Ovid and the gi...,"The mind, conscious of rectitude, laughed to s...",Metamorphoses of Publius
4,0.7,"Based on the given quote ""Every lover is a sol...",Every lover is a soldier.,Metamorphoses of Publius
5,-1,"The given text ""Resist beginnings; the remedy ...",Resist beginnings; the remedy comes too late w...,Remedia Amoris
6,0.9,Based on the provided quote the punctuation st...,"Let him who loves, where love success may find...",Metamorphoses of Publius
7,0.9,On a scale of 0 to 1 I would assess the likeli...,We take no pleasure in permitted joys. But wha...,Metamorphoses of Publius
8,0.8,Likelihood: 0.8\n\nReasoning: The punctuation ...,"Who is allowed to sin, sins less.",Metamorphoses of Publius
9,0.8,"Based on the provided text ""Time flies death u...","Time flies, death urges, heaven's revolving wh...",Metamorphoses of Publius


In [8]:
scored_quotes_df

,score,reason,quote,work
0,0.9,Based on the provided quote the likelihood tha...,Love yields to business. If you seek a way out...,Metamorphoses of Publius
1,0.9,Based on the provided quote I would assess the...,The gods behold all righteous actions.,Metamorphoses of Publius
2,0.8,Confidence Score: 0.8\n\nExplanation: The writ...,There is a god within us.,Metamorphoses of Publius
3,0.9,Based on the provided quote by Ovid and the gi...,"The mind, conscious of rectitude, laughed to s...",Metamorphoses of Publius
4,0.7,"Based on the given quote ""Every lover is a sol...",Every lover is a soldier.,Metamorphoses of Publius
5,-1,"The given text ""Resist beginnings; the remedy ...",Resist beginnings; the remedy comes too late w...,Remedia Amoris
6,0.9,Based on the provided quote the punctuation st...,"Let him who loves, where love success may find...",Metamorphoses of Publius
7,0.9,On a scale of 0 to 1 I would assess the likeli...,We take no pleasure in permitted joys. But wha...,Metamorphoses of Publius
8,0.8,Likelihood: 0.8\n\nReasoning: The punctuation ...,"Who is allowed to sin, sins less.",Metamorphoses of Publius
9,0.8,"Based on the provided text ""Time flies death u...","Time flies, death urges, heaven's revolving wh...",Metamorphoses of Publius


### Assessment of Retrieval Accuracy

The below uses current pipeline for 2 reasons:
1) To assess the hulicination affect against current pipeline
2) To assess scoring variability

In [9]:
import re

non_ovid_generated_quotes = [
    "One man's meat is another man's poison.",
    "Fortune favors the bold.",
    "Wherever there is a human being, there is an opportunity for a kindness.",
    "Love is a kind of warfare.",
    "One man's meat is another man's poison.",
    "To be loved, be lovable.",
]


# Define the regex pattern
pattern = r"(\d+\.\d+)"


scores = []
strings = []
for q in non_ovid_generated_quotes:
    r = chain.invoke({"text": q})
    r = " ".join(r)
    strings.append(r)
    match = re.search(pattern, r)
    score = match[0] if match else -1
    scores.append(score)
    print(r)

pd.DataFrame(
    data={
        "questionable_quote": non_ovid_generated_quotes,
        "authorship_match_score": scores,
        "is_original": [0, 0, 0, 1, 0, 1],
    }
)

On a scale of 0 to 1 with 0 indicating low confidence and 1 indicating high confidence I would assess the likelihood that the given text is written by the same author as the provided reference to be 0. The quote "One man's meat is another man's poison" does not exhibit any similarities in terms of punctuation style special characters style acronyms and abbreviations writing style expressions and idioms tone and mood sentence structure or any other relevant aspect to the quotes by Ovid. Therefore I have low confidence that the given text is written by the same author.
Confidence Score: 0.6

Explanation: The given text does not exhibit any specific characteristics or writing style that can be directly attributed to Ovid. However the tone of the statement aligns with Ovid's style of expressing wisdom and aphorisms. The sentence structure is simple and concise which is also consistent with Ovid's writing style. Overall while there are no clear indicators there is a moderate likelihood that

,questionable_quote,authorship_match_score,is_original
0,One man's meat is another man's poison.,-1,0
1,Fortune favors the bold.,0.6,0
2,"Wherever there is a human being, there is an o...",0.3,0
3,Love is a kind of warfare.,0.6,1
4,One man's meat is another man's poison.,0.3,0
5,"To be loved, be lovable.",0.8,1


In [10]:
scored_quotes_df[scored_quotes_df["score"].astype(float) < 0.5]["quote"].tolist()

['Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.']

In [11]:
from itertools import product

# Compare to quotes
generated_quotes = scored_quotes_df[scored_quotes_df["score"].astype(float) > 0.5][
    "quote"
].tolist()

# Read in existing quotes
df = pd.read_json("./../ovid_quotes.json")

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Process each quote and calculate similarity scores
similarity_data = []

for generated_quote, actual_quote in product(generated_quotes, df["Quote"]):
    # Process the quotes with spaCy
    doc_generated = nlp(generated_quote)
    doc_actual = nlp(actual_quote)

    # Calculate similarity score
    similarity_score = doc_generated.similarity(doc_actual)

    # Append data to the list
    similarity_data.append(
        {
            "generated_quotes": generated_quote,
            "actual_quotes": actual_quote,
            "similarity_score": similarity_score,
        }
    )

# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data).sort_values(
    "similarity_score", ascending=False
)
display(similarity_df.head(2))

# Seems like there might be a magic number similarity score < 0.65
print("unique quutes")

similiar_quotes = similarity_df[similarity_df["similarity_score"] > 0.65][
    "generated_quotes"
].unique()

# Filter out the similar quotes
new_quotes = similarity_df[~similarity_df["generated_quotes"].isin(similiar_quotes)][
    "generated_quotes"
].unique()

# Get the works for the new quotes
new_quote_works = []
for q in new_quotes:
    for quote, work in quotes:
        if q == quote:
            new_quote_works.append(work)
df_new_quotes = pd.DataFrame(
    list(zip(*(new_quotes, new_quote_works))), columns=["Quote", "Work"]
)
df_new_quotes["Quote in Latin"] = None
df_new_quotes

/var/folders/qq/gdppvj912kv3ds7_xnf0q_fc0000gn/T/ipykernel_6724/1111017412.py:23: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc_generated.similarity(doc_actual)


,generated_quotes,actual_quotes,similarity_score
239,"Who is allowed to sin, sins less.","Who is allowed to sin, sins less.",1.0
164,"Let him who loves, where love success may find...","Let him who loves, where love success may find...",1.0


unique quutes


,Quote,Work,Quote in Latin


In [12]:
new_quotes

array([], dtype=object)

In [40]:
print("highest score comparison:")
idx = 54
print("GENERATED:", similarity_df.iloc[54]["generated_quotes"])
print("ACTUAL:", similarity_df.iloc[54]["actual_quotes"])

highest score comparison:
GENERATED: "Everything changes, nothing perishes." - Metamorphoses
ACTUAL: The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”


### Storage of Additional Quotes

In [68]:
!ls ./../

DOCS.md              config.ini           ovid_quotes.json
ProjectManagement.md data                 requirements.txt
README.md            myscript.sh          venv
__pycache__          notebooks
app                  ovid_quotes.csv


In [76]:
# After quotes works have passed the assessment bars above
df = pd.concat([df, df_new_quotes]).reset_index(drop=True)
df.to_json("./../ovid_quotes.json")